In [1]:
import spacy
import pandas as pd
from nltk import edit_distance
import tqdm

2023-09-27 22:43:27.542815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nlp = spacy.load("it_core_news_lg")

In [8]:
df = pd.read_excel("ita_morph_pos.xlsx")
df

,Unnamed: 0,fqTot,gramCat,lemma,word,Phones,morph,pos,Gender,Number,...,Person,Tense,VerbForm,Clitic,PronType,Foreign,Degree,NumType,Definite,Poss
0,0,50518,P,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,5219,P IN P@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1544,P IN B@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,272,P IN C@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,234,S,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73805,119992,28,E,zurigo,zurigo,ZuriGo,NaN,PROPN,0,0,...,0,0,0,0,0,0,0,0,0,0
73806,119994,2,E,zurletti,zurletti,Zurletti,Gender=Masc|Number=Plur|PronType=Ind,PRON,Masc,Plur,...,0,0,0,0,Ind,0,0,0,0,0
73807,119995,5,E,zurli,zurli,Zurli,Clitic=Yes|Gender=Masc|Number=Plur|Person=3|Pr...,VERB,Masc,Plur,...,3,0,Inf,Yes,Prs,0,0,0,0,0
73808,119996,2,E IN E@,zurli',zurli',Zurli,Clitic=Yes|Gender=Masc|Number=Plur|Person=3|Pr...,VERB,Masc,Plur,...,3,0,Inf,Yes,Prs,0,0,0,0,0


In [35]:
memo = {}

def max_prefix(s1, s2):
    if s1 == s2:
        return 0
    i = 0
    l1, l2 = len(s1), len(s2)
    try:
        while s1[i] == s2[i]:
            i += 1
        return i
    except:
        return i
    
i = 0
def find_closest_lemma(goal_lemma):
    if goal_lemma in memo:
        return memo[goal_lemma]
    # takes too long
    dist = df['Phones'].apply(lambda l: -max_prefix(l, goal_lemma))
    mins = df[dist == dist.min()]
    return mins.loc[mins['Phones'].apply(len).idxmin()]
#     print(mins)
#     memo[goal_lemma] = mins.loc[mins['Phones'].apply(len).idxmin(), 'lemma']
#     return memo[goal_lemma]
    
    
# df["actual_lemma"] = df["Phones"].apply(find_closest_lemma)
# df.to_excel("ita_deriv2.xlsx")

In [20]:
def extra(word, lemma):
    return word[len(lemma):]

In [36]:
# df['actual_lemma_pos'] = df['actual_lemma'].apply(lambda i: df[df['lemma'] == i].iloc[0]['gramCat'])
df['actual_lemma_pos'] = ''
df['actual_lemma_phones'] = ''
df['extra'] = ''
for idx, row in tqdm.tqdm(df.iterrows()):
    a = find_closest_lemma(row["Phones"])
#     print(row["Phones"], a)
    df.loc[idx, 'actual_lemma'] = a['lemma']
#     d = df[df['lemma'] == a].iloc[0]
    df.loc[idx, 'actual_lemma_pos'] = a['gramCat']
    df.loc[idx, 'actual_lemma_phones'] = a['Phones']
    df.loc[idx, 'extra'] = extra(row['Phones'], a['Phones'])
df

420it [00:13, 31.93it/s]


KeyboardInterrupt: 

In [37]:
df

,Unnamed: 0,fqTot,gramCat,lemma,word,Phones,morph,pos,Gender,Number,...,PronType,Foreign,Degree,NumType,Definite,Poss,actual_lemma,actual_lemma_pos,extra,actual_lemma_phones
0,0,50518,P,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,a,P,,ad
1,1,5219,P IN P@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,a,P,,ad
2,2,1544,P IN B@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,a,P,,ad
3,3,272,P IN C@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,a,P,,ad
4,4,234,S,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,a,P,,ad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73805,119992,28,E,zurigo,zurigo,ZuriGo,NaN,PROPN,0,0,...,0,0,0,0,0,0,zucca,,,
73806,119994,2,E,zurletti,zurletti,Zurletti,Gender=Masc|Number=Plur|PronType=Ind,PRON,Masc,Plur,...,Ind,0,0,0,0,0,zucca,,,
73807,119995,5,E,zurli,zurli,Zurli,Clitic=Yes|Gender=Masc|Number=Plur|Person=3|Pr...,VERB,Masc,Plur,...,Prs,0,0,0,0,0,zucca,,,
73808,119996,2,E IN E@,zurli',zurli',Zurli,Clitic=Yes|Gender=Masc|Number=Plur|Person=3|Pr...,VERB,Masc,Plur,...,Prs,0,0,0,0,0,zucca,,,


In [38]:
df.to_excel("ita_deriv5.xlsx")

In [23]:
for idx, row in tqdm.tqdm(df.iterrows()):
    if row['extra']:
        print(row['gramCat'], row['word'], row['actual_lemma'], row['actual_lemma_pos'], row['extra'], end='')
        input()

0it [00:00, ?it/s]

S abbagli abba E LLi

28it [01:15,  2.71s/it]


KeyboardInterrupt: Interrupted by user

In [53]:
df = pd.read_excel("ita_deriv_6.xlsx")

# suffixes = ['eggi', 'ific', 'izz', 'aggi', 'aggin', 'ai', 'aiol', 'anz', 'enz', 'eri', 'ezz', 'i\'', 'ier', 'ifici', 'in', 'ism', 'esim', 'ist', 'ita\'', 'eta\'', 'ta\'', 'ituden', 'etudin', 'izi', 'uzi', 'ment', 'toi', 'tor', 'sor', 'tric', 'tori', 'ur', 'zion', 'sion', 'ace', 'al', 'ar', 'ari', 'atori', 'bil', 'es', 'an', 'in', 'asc', 'esc', 'evol', 'ic', 'ier', 'ifer', 'ific', 'efic', 'il', 'iv', 'izi', 'os', 'str']

suffixes = {
    "E V event/process": ["izzare", "ificare"],
    "E E pertaining to/follwoing": ["ismo", "esimo"],
    "E E actively involved in": ["ista"],
    "E E quantity of": ["aglia", "ame", "ume", "ata"],
    "E E hit": ["ata"],
    "E E place with": ["eta", "eto"],
    "E E place for": ["ile"],
    "E E containing": ["ato"],
    "E E feminine": ["essa"],
    "E G related to": ["ale", "torio", "ino", "eo", "are", "ano", 
                            "esco", "aso", "ico", "istico", "ivo", 
                            "izio", "'ile", "oso"],
    "E G similar to": ["aceo", "ino", "oide", "igno", "are"],
    "E G belonging to": ["ardo", "asco", "ico", "ano"],
    "E G inhabitant of": ["ano", "ino", "etano", "itano", "aneo", "eo", "ese", "ate", 
                         "ota", "are", "esco", "ico", "eno"],
    "V N effect of": ["ata", "ione", "ment", "aggi", "ura", "nza", "io", "eria"],
    "V N state of": ["ato", "nza", "ia", "io", "ore"],
    "V N related to": ["toio", "toia", "torio"],
    "V N the point of": ["ando"],
    "V N agent of": ["tore", "trice", "iere", "iero", "ante", "aro"],
    "V G referring to": ["evole", "ivo", "torio", "ario"],
    "V V grade of": ["ellare", "arellare", "erellare", "icchiare", "acchiare", "ettare"],
    "G N quality of": ["ita'", "udine", "ezza", "izia", "igia", "aggine", "ia"],
    "G V change status of": ["izzare", "ificare"],
    "G N phenomenon of": ["ismo", "esimo"],
    "G N actively involved in": ["ista"],
    "G G manner of": []
}

# df['actual_lemma'] = ''
df['derivational'] = ''
df['process'] = ''
df['reason'] = ''

for idx, row in tqdm.tqdm(df.iterrows()):
    w = row['word']
    try:
        for typ, suffs in suffixes.items():
            for s in suffs:
                if s in row['extra']:
#                     goal_lemma = w[:w.index(s)]
#                     if lemma := find_closest_lemma(goal_lemma):
        #                 print(goal_lemma, lemma)
        #                 input()
#                         df.loc[idx, 'actual_lemma'] = lemma
#                         df.loc[idx, 'derivational'] = s
                    t1, t2, *reason = typ.split()
                    if row["gramCat"] == t2: # todo use t1
                        df.loc[idx, 'process'] = f"{t1} => {t2}"
                        df.loc[idx, 'reason'] = ' '.join(reason)
#                     break
#         for p in prefixes:
#             ...
    except Exception as e:
        print(e, idx, w) #, row)

647it [00:00, 3444.44it/s]

argument of type 'float' is not iterable 0
argument of type 'float' is not iterable 1
argument of type 'float' is not iterable 2
argument of type 'float' is not iterable 3
argument of type 'float' is not iterable 4
argument of type 'float' is not iterable 5
argument of type 'float' is not iterable 6
argument of type 'float' is not iterable 7
argument of type 'float' is not iterable 8
argument of type 'float' is not iterable 9
argument of type 'float' is not iterable 10
argument of type 'float' is not iterable 11
argument of type 'float' is not iterable 12
argument of type 'float' is not iterable 13
argument of type 'float' is not iterable 14
argument of type 'float' is not iterable 15
argument of type 'float' is not iterable 16
argument of type 'float' is not iterable 17
argument of type 'float' is not iterable 18
argument of type 'float' is not iterable 19
argument of type 'float' is not iterable 20
argument of type 'float' is not iterable 21
argument of type 'float' is not iterable 2

1441it [00:00, 3711.91it/s]

argument of type 'float' is not iterable 966
argument of type 'float' is not iterable 967
argument of type 'float' is not iterable 968
argument of type 'float' is not iterable 996
argument of type 'float' is not iterable 998
argument of type 'float' is not iterable 999
argument of type 'float' is not iterable 1000
argument of type 'float' is not iterable 1001
argument of type 'float' is not iterable 1005
argument of type 'float' is not iterable 1006
argument of type 'float' is not iterable 1009
argument of type 'float' is not iterable 1010
argument of type 'float' is not iterable 1011
argument of type 'float' is not iterable 1012
argument of type 'float' is not iterable 1018
argument of type 'float' is not iterable 1019
argument of type 'float' is not iterable 1020
argument of type 'float' is not iterable 1021
argument of type 'float' is not iterable 1022
argument of type 'float' is not iterable 1023
argument of type 'float' is not iterable 1024
argument of type 'float' is not iterable

2870it [00:00, 4229.53it/s]

...

argument of type 'float' is not iterable 71019
argument of type 'float' is not iterable 71020
argument of type 'float' is not iterable 71088
argument of type 'float' is not iterable 71094
argument of type 'float' is not iterable 71095
argument of type 'float' is not iterable 71101
argument of type 'float' is not iterable 71102
argument of type 'float' is not iterable 71103
argument of type 'float' is not iterable 71104
argument of type 'float' is not iterable 71105
argument of type 'float' is not iterable 71106
argument of type 'float' is not iterable 71107
argument of type 'float' is not iterable 71108
argument of type 'float' is not iterable 71114
argument of type 'float' is not iterable 71115
argument of type 'float' is not iterable 71117
argument of type 'float' is not iterable 71119
argument of type 'float' is not iterable 71120
argument of type 'float' is not iterable 71121
argument of type 'float' is not iterable 71122
argument of type 'float' is not iterable 71123
argument of t

72726it [00:16, 5406.86it/s]

argument of type 'float' is not iterable 72122
argument of type 'float' is not iterable 72123
argument of type 'float' is not iterable 72124
argument of type 'float' is not iterable 72125
argument of type 'float' is not iterable 72126
argument of type 'float' is not iterable 72127
argument of type 'float' is not iterable 72128
argument of type 'float' is not iterable 72131
argument of type 'float' is not iterable 72132
argument of type 'float' is not iterable 72134
argument of type 'float' is not iterable 72135
argument of type 'float' is not iterable 72137
argument of type 'float' is not iterable 72138
argument of type 'float' is not iterable 72139
argument of type 'float' is not iterable 72140
argument of type 'float' is not iterable 72141
argument of type 'float' is not iterable 72142
argument of type 'float' is not iterable 72143
argument of type 'float' is not iterable 72145
argument of type 'float' is not iterable 72146
argument of type 'float' is not iterable 72165
argument of t

73810it [00:16, 4503.78it/s]

argument of type 'float' is not iterable 73189
argument of type 'float' is not iterable 73190
argument of type 'float' is not iterable 73196
argument of type 'float' is not iterable 73197
argument of type 'float' is not iterable 73198
argument of type 'float' is not iterable 73204
argument of type 'float' is not iterable 73205
argument of type 'float' is not iterable 73206
argument of type 'float' is not iterable 73207
argument of type 'float' is not iterable 73208
argument of type 'float' is not iterable 73209
argument of type 'float' is not iterable 73212
argument of type 'float' is not iterable 73213
argument of type 'float' is not iterable 73240
argument of type 'float' is not iterable 73244
argument of type 'float' is not iterable 73245
argument of type 'float' is not iterable 73246
argument of type 'float' is not iterable 73247
argument of type 'float' is not iterable 73248
argument of type 'float' is not iterable 73249
argument of type 'float' is not iterable 73250
argument of t

In [1]:
nlp.pipeline

NameError: name 'nlp' is not defined

In [41]:
# prefixes = ['in', 'inn', 'im', 'ir', 'il']
# df['actual_lemma_pref'] = ''
# df['derivational_pref'] = ''

# for idx, row in tqdm.tqdm(df.iterrows()):
#     w = row['word']
#     try:
#         for p in prefixes:
#             if w.startswith(p):
#                 goal_lemma = w[len(p):]
#                 if lemma := find_closest_lemma(goal_lemma):
# #                     print(goal_lemma, lemma)
# #                     input()
#                     df.loc[idx, 'actual_lemma_pref'] = lemma
#                     df.loc[idx, 'derivational_pref'] = p
#                 break
#             elif w.startswith('a'):
#                 ...
#         for p in prefixes:
#             ...
#     except Exception as e:
#         print(e, idx) #, row)

29721it [00:01, 28936.34it/s]

'bool' object has no attribute 'startswith' 25507 Unnamed: 0           41364
fqTot                   43
gramCat                  G
lemma                falso
word                 False
Phones               False
morph                  NaN
pos                    NaN
Gender                   0
Number                   0
Mood                     0
Person                   0
Tense                    0
VerbForm                 0
Clitic                   0
PronType                 0
Foreign                  0
Degree                   0
NumType                  0
Definite                 0
Poss                     0
actual_lemma              
derivational              
actual_lemma_pref         
derivational_pref         
Name: 25507, dtype: object


73810it [00:03, 23597.17it/s]


In [42]:
df.to_excel("ita_deriv.xlsx")

In [44]:
nlp = spacy.load("it_core_news_lg", disable=['parser', 'ner'])
df['spacy_lemma'] = ''
for idx, row in tqdm.tqdm(df.iterrows()):
    df.loc[idx, 'spacy_lemma'] = nlp(row["word"])[0].lemma_
#     print(nlp(row["word"])[0].lemma_)
#     input()
# df['spacy_lemma'] = df['word'].apply(lambda i: nlp(i)[0].lemma_)

73810it [02:25, 507.45it/s]


In [45]:
import nltk
from nltk.stem import SnowballStemmer
stemmer = nltk.stem.SnowballStemmer('italian')
df['nltk_lemma'] = ''
for idx, row in tqdm.tqdm(df.iterrows()):
    df.loc[idx, 'nltk_lemma'] = stemmer.stem(row["word"])

73810it [00:09, 7650.11it/s]


In [54]:
df.to_excel("ita_deriv_8.xlsx")